# Pump + Tubing Flow Model Documentation

## 1. Ideal gas law
In a sealed loop at constant temperature $T$:

$$
p V = N k_B T = m R T
$$

where:
- $p$: pressure (Pa, absolute)  
- $V$: loop volume (m³)  
- $N$: number of molecules  
- $m$: total gas mass (kg)
- $R$: specific gas constant (J/kg/K)  

👉 Since $m, V, T$ are fixed, the **mean absolute pressure** is conserved:

$$
p_{\text{mean}} = \frac{m R T}{V}.
$$

---

## 2. Pump curve (overpressure mode)
Datasheet gives volumetric flow vs. pressure rise $\Delta p$ (bar):

$$
Q(\Delta p) = Q_0 - s\,\Delta p
$$

- $Q$: volumetric flow at pump inlet conditions (L/min)  
- $Q_0$: free flow at $\Delta p = 0$  
- $s$: slope (L/min per bar)  

Converted to **mass flow**:

$$
\dot m_{\text{pump}}(\Delta p, p_\text{in}) = \rho_\text{in}\, Q(\Delta p)
$$

with inlet density

$$
\rho_\text{in} = \frac{p_\text{in}}{R T}.
$$

---

## 3. Pump curve (vacuum mode)
Datasheet also provides maximum flow vs. **inlet absolute pressure** $p_\text{in}$:

$$
Q_{\max}(p_\text{in}) \quad \text{(from curve)}.
$$

Effective pump volumetric flow is the **minimum** of the two limits:

$$
Q_{\text{pump, eff}}(\Delta p, p_\text{in}) =
\min\big(Q(\Delta p),\; Q_{\max}(p_\text{in})\big).
$$

Then:

$$
\dot m_{\text{pump}} = \rho_\text{in}\, Q_{\text{pump, eff}}.
$$

---

## 4. Tube model (isothermal compressible flow)
Darcy–Weisbach equation:

$$
\Delta p = f \frac{L}{D} \frac{\rho v^2}{2}.
$$

But in compressible isothermal flow, $Q(x)$ varies while $\dot m$ is constant.  
Integrating with $\rho = p / (RT)$ yields:

$$
p_1^2 - p_2^2 = \frac{f L_\text{eff}}{D}\;\frac{R T}{A^2}\;\dot m^2
$$

- $p_1, p_2$: inlet/outlet absolute pressures (Pa)  
- $f$: Darcy friction factor  
- $L_\text{eff} = L + \tfrac{K D}{f}$: length + equivalent length of minor losses  
- $D$: tube inner diameter (m)  
- $A = \pi D^2/4$: cross-sectional area (m²)  
- $\dot m$: mass flow (kg/s)

---

## 5. Friction factor $f$
Reynolds number:

$$
Re = \frac{\dot m D}{\mu A}
$$

where $\mu$ = dynamic viscosity (Pa·s).

Friction factor models:
- **Laminar** ($Re < 2300$):

$$
f = \frac{64}{Re}
$$

- **Turbulent, smooth** (Blasius):

$$
f = 0.316\, Re^{-1/4}
$$

- **General explicit** (Haaland, with roughness $\varepsilon$):

$$
\frac{1}{\sqrt{f}} = -1.8 \log_{10}\left[\left(\frac{\varepsilon/D}{3.7}\right)^{1.11} + \frac{6.9}{Re}\right]
$$

---

## 6. Closed-loop solver
Mean absolute pressure is fixed:

$$
p_\text{in} = p_\text{mean} - \tfrac{1}{2}\Delta p, \quad
p_\text{out} = p_\text{mean} + \tfrac{1}{2}\Delta p
$$

Solve for $\Delta p$ such that:

$$
\dot m_{\text{pump}}(\Delta p, p_\text{in}) = \dot m_{\text{tube}}(p_\text{out}, p_\text{in})
$$

---

## 7. Open-case solver
With fixed boundaries:

$$
p_\text{in} = \text{given}, \quad p_\text{out} = \text{given}
$$

then

$$
\dot m = \min\Big( \dot m_{\text{pump}}(\Delta p, p_\text{in}),\; \dot m_{\text{tube}}(p_\text{out}, p_\text{in}) \Big)
$$

---

## 8. Useful conversions
- Volumetric flow:

$$
Q\,[\mathrm{m^3/s}] = \frac{Q\,[\mathrm{L/min}]}{1000 \cdot 60}
$$

- Average volumetric flow at mean density:

$$
Q_{\text{avg}} = \frac{\dot m}{\rho_{\text{mean}}}, \quad
\rho_{\text{mean}} = \frac{p_{\text{mean}}}{R T}
$$

# Deriving the isothermal compressible pipe relation (and how it ties into the pump model)

This note derives the isothermal, compressible pressure–flow relation used in the code:

$$
p_1^2 - p_2^2 \;=\; \frac{f\,L_\text{eff}}{D}\;\frac{R T}{A^2}\;\dot m^{\,2}
\quad\text{(Darcy $f$; isothermal ideal gas)}
$$

and shows how the **pump curve** and **closed-loop mean pressure** fit together.

---

## 1) Setup and assumptions

- Straight tube, inner diameter $D$, cross-section $A=\pi D^2/4$, length $L$.
- Steady 1D flow of an ideal gas, **isothermal** at temperature $T$ (good with water-cooled head).
- **Low Mach** (subsonic), no shaft work inside the tube, negligible elevation change.
- Only wall friction contributes significantly to $\mathrm d p/\mathrm d x$.
- **Mass flow is constant** along the tube: $\dot m = \rho v A$.

We use the **Darcy–Weisbach** wall shear model:

$$
-\frac{\mathrm d p}{\mathrm d x} \;=\; f\,\frac{\rho v^2}{2D}
$$

where $f$ is the **Darcy** friction factor (function of Reynolds number and roughness).

For an ideal gas at constant $T$:
$$
\rho(x) \;=\; \frac{p(x)}{R\,T}.
$$

---

## 2) Express everything in terms of $\dot m$ and $p(x)$

Velocity $v(x)$ in terms of $\dot m$ and $p$:

$$
v(x) \;=\; \frac{\dot m}{\rho A} \;=\; \frac{\dot m\,R T}{p(x)\,A}.
$$

Hence
$$
\rho v^2
= \frac{p}{RT}\,\left(\frac{\dot m\,RT}{pA}\right)^2
= \frac{(RT)\,\dot m^2}{A^2}\;\frac{1}{p}.
$$

Substitute into Darcy–Weisbach:

$$
-\frac{\mathrm d p}{\mathrm d x}
= f\,\frac{1}{2D}\;\frac{(RT)\,\dot m^2}{A^2}\;\frac{1}{p}.
$$

Rearrange:

$$
-\,p\,\frac{\mathrm d p}{\mathrm d x}
= \frac{f}{2D}\;\frac{R T}{A^2}\;\dot m^2.
$$

---

## 3) Integrate along the pipe
For the ease of rewriting:
$$
C \equiv \frac{f}{2D}\frac{RT \dot m^2}{A^2}
$$

So:
$$
-p \mathrm d p = \frac{f}{2D}\frac{RT \dot m^2}{A^2} \mathrm d x = C \mathrm dx 
$$

Integrate from inlet ($x=0$, $p=p_1$) to outlet ($x=L$, $p=p_2$). Assume, for the moment, $f$ is **uniform** along the pipe (we’ll revisit this).
Start with the general case from $p_1$ to $p(x)$:

$$
-\frac{1}{2}p^2 \big|_{p_1}^{p(x)} = K x \big|_{0}^{x} \rightarrow -\frac{1}{2} (p^2(x)-p_1^2) = Kx
$$

$$
p^2(x) - p_1^2 = - 2Kx = -\frac{f}{D}\frac{RT \dot m^2}{A^2}x
$$

For the the entire pipe length (L, p_2)
$$
p^2(L) - p_1^2 = p_2^2 - p_1^2 = -\frac{f}{D}\frac{RT \dot m^2}{A^2}L
$$

Which finally yields:
$$
\boxed{
p_1^2- p_2^2  = -\frac{fL}{D}\frac{RT \dot m^2}{A^2}
}
$$

---

## 4) Minor losses as equivalent length

If the pipe segment includes fittings/valves/elbows with total *loss coefficient* $K$,
you can fold them in as an **equivalent length**:

$$
L_\text{eff} \;=\; L \;+\; \frac{K\,D}{f}.
$$

Using $L_\text{eff}$ in the boxed relation captures both distributed and minor losses:

$$
p_1^2 - p_2^2
= \frac{4 f\,L_\text{eff}}{D}\;\frac{R T}{A^2}\;\dot m^2.
$$

Note that $L_\text{eff}$ depends on $f$, and $f$ depends on Reynolds number, which depends on $\dot m$ — this is why we solve **iteratively** in code.

---

## 5) Reynolds number and friction factor models

Reynolds number written in terms of $\dot m$:

$$
Re = \frac{\rho v D}{\mu} = \frac{\dot m\,D}{\mu\,A}.
$$

Typical $f(Re,\varepsilon/D)$ relations (Darcy $f$):
- **Laminar ($Re<2300$)**: $f = \dfrac{64}{Re}$.
- **Turbulent, smooth (Blasius)**: $f \approx 0.316\,Re^{-1/4}$.
- **General explicit (Haaland)**:
  $$
  \frac{1}{\sqrt{f}} = -1.8\log_{10}\!\left[\left(\frac{\varepsilon/D}{3.7}\right)^{1.11} + \frac{6.9}{Re}\right],
  $$
  which we use in the code for robustness.

Because $f$ depends on $Re(\dot m)$, and minor losses use $L_\text{eff}=L+K D/f$, the pressure–flow relation is **implicit** in $\dot m$ and solved by bisection/fixed-point.

---

## 6) Open run vs. closed loop

### Open run (boundaries given)
If both end pressures are specified (absolute):
$$
p_\text{in},\; p_\text{out}\;\text{given,} \quad \Delta p = p_\text{out}-p_\text{in}.
$$

- **Tube-only** mass flow from the boxed relation (iterating $f$),
- **Pump-only** mass flow from the **pump curve** (below) using inlet density,
- The **achievable** $\dot m$ is the **minimum** of the two.

### Closed loop (sealed, isothermal)
Total gas is fixed, so the **volume-average pressure** is conserved:

$$
p_\text{mean} \equiv \frac{1}{V}\!\int_V p(\mathbf r)\,\mathrm dV = \frac{m R T}{V}\;\text{(constant)}.
$$

We model the pump creating a symmetric spread about the mean:

$$
p_\text{in} = p_\text{mean} - \frac{\Delta p}{2}, \qquad
p_\text{out} = p_\text{mean} + \frac{\Delta p}{2}.
$$

Solve for $\Delta p$ such that
$$
\dot m_\text{pump}(\Delta p, p_\text{in}) \;=\; \dot m_\text{tube}(p_\text{out}, p_\text{in}).
$$

The reported “average volumetric flow” in the code is
$$
Q_{\text{avg}} \;=\; \frac{\dot m}{\rho_{\text{mean}}},\qquad
\rho_{\text{mean}}=\frac{p_{\text{mean}}}{R T}.
$$

---

## 7) Pump curves (overpressure and vacuum)

**Overpressure (datasheet)**: a straight-line fit is often adequate over 0–3 bar:

$$
Q(\Delta p) = Q_0 - s\,\Delta p
\quad\text{(L/min at pump inlet conditions, $\Delta p$ in bar)}.
$$

Convert to mass flow using **inlet density**:
$$
\dot m_\text{pump,over}(\Delta p, p_\text{in}) = \rho_\text{in}\,Q(\Delta p),
\qquad
\rho_\text{in} = \frac{p_\text{in}}{R T}.
$$

**Vacuum limit (datasheet)**: maximum volumetric flow vs. **inlet absolute pressure**
with the outlet vented (often tabulated or digitized from the plot):
$$
Q_{\max}(p_\text{in}) \;\;\Rightarrow\;\; \dot m_\text{pump,vac} = \rho_\text{in}\,Q_{\max}(p_\text{in}).
$$

**Effective pump mass flow** is the tighter of the two:
$$
\dot m_\text{pump}(\Delta p, p_\text{in}) =
\min\!\Big(\,\dot m_\text{pump,over}(\Delta p, p_\text{in})\;,\;\dot m_\text{pump,vac}(p_\text{in})\Big).
$$

This guarantees that as $p_\text{in}$ approaches the **ultimate vacuum** on the datasheet,
the modeled pump flow correctly collapses toward **zero**.

---

## 8) Practical notes

- The derivation above assumes **$f$ uniform** while integrating; in practice we
  handle the $f(\dot m)$ coupling numerically (update $f$ from $Re(\dot m)$ on each
  trial and, if minor losses exist, update $L_\text{eff}$ consistently).
- Units in code:
  pressures in **Pa** internally (inputs/outputs often shown in **bar abs** for convenience);
  $T$ in K, $R$ in J/kg/K, $\mu$ in Pa·s, lengths in m, flows in kg/s or L/min.
- For stainless steel tubing roughness, a typical starting value is
  $\varepsilon \sim 10^{-6}\,\text{m}$; PTFE is much smoother (often set $\varepsilon\approx 0$).

---